# ***LLaMA-3.2-3B (or other models) Eval on GSM8K***

In [ ]:
import os
os.environ["HF_TOKEN"] = ###

In [ ]:
import os
os.environ["WANDB_API_KEY"] = ###

**Change the following to desired parent folder. Google Drive should be mounted.**

In [3]:
cd drive/MyDrive/Colab\ Notebooks

/content/drive/MyDrive/Colab Notebooks


In [4]:
!pip install transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install torch
!pip install --upgrade jax
!pip install jaxlib

In [ ]:
import transformers
print(transformers.__version__)

4.48.3


# **Dataset Prep**

In [24]:
from datasets import load_dataset

gsm8k_train = load_dataset("openai/gsm8k", "main", split="train")
gsm8k_test = load_dataset("openai/gsm8k", "main", split="test")

In [ ]:
# Create place to store llama 405B outputs
default_values = ["None"] * len(gsm8k_train)
gsm8k_kd_train = gsm8k_train.add_column("405B_answer", default_values)
gsm8k_kd_train = gsm8k_kd_train.add_column("405B_predictions", default_values)
gsm8k_kd_train = gsm8k_kd_train.add_column("405B_logprobs", default_values)

In [7]:
default_values = ["None"] * len(test_dataset)
test_405 = test_dataset.add_column("405B_answer", default_values)

Flattening the indices:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
print(gsm8k_kd_train[0])

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72', '405B_answer': 'None', '405B_predictions': 'None', '405B_logprobs': 'None'}


In [8]:
import random
from tqdm import tqdm
import re

In [ ]:
# Take 400 random samples of dataset
gsm8k_kd_sample = gsm8k_kd_train.shuffle(seed=42).select(range(400))

In [ ]:
# Take 400 more random samples of dataset
gsm8k_kd_sample_new = gsm8k_kd_train.shuffle(seed=42).select(range(400, 800))

In [ ]:
print(gsm8k_kd_sample_new)

Dataset({
    features: ['question', 'answer', '405B_answer', '405B_predictions', '405B_logprobs'],
    num_rows: 400
})


In [9]:
ANS_RE = re.compile(r"#### (\-?[0-9\.\,]+)")
INVALID_ANS = "[invalid]"

N_SHOT = 8
COT_FLAG = True
DEBUG = False
ANSWER_TRIGGER = "The answer is: "

In [19]:
# Demo Examples for CoT
def create_demo_text(n_shot=8, cot_flag=True):
    question, chain, answer = [], [], []
    question.append(
        "There are 15 trees in the grove. "
        "Grove workers will plant trees in the grove today. "
        "After they are done, there will be 21 trees. "
        "How many trees did the grove workers plant today?"
    )
    chain.append(
        "There are 15 trees originally. "
        "Then there were 21 trees after some more were planted. "
        "So there must have been 21 - 15 = 6."
    )
    answer.append("6")

    question.append(
        "If there are 3 cars in the parking lot and 2 more cars arrive, "
        "how many cars are in the parking lot?"
    )
    chain.append("There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5.")
    answer.append("5")

    question.append(
        "Leah had 32 chocolates and her sister had 42. If they ate 35, "
        "how many pieces do they have left in total?"
    )
    chain.append(
        "Originally, Leah had 32 chocolates. "
        "Her sister had 42. So in total they had 32 + 42 = 74. "
        "After eating 35, they had 74 - 35 = 39."
    )
    answer.append("39")

    question.append(
        "Jason had 20 lollipops. He gave Denny some lollipops. Now Jason "
        "has 12 lollipops. How many lollipops did Jason give to Denny?"
    )
    chain.append(
        "Jason started with 20 lollipops. Then he had 12 after giving some "
        "to Denny. So he gave Denny 20 - 12 = 8."
    )
    answer.append("8")

    question.append(
        "Shawn has five toys. For Christmas, he got two toys each from his "
        "mom and dad. How many toys does he have now?"
    )
    chain.append(
        "Shawn started with 5 toys. If he got 2 toys each from his mom and "
        "dad, then that is 4 more toys. 5 + 4 = 9."
    )
    answer.append("9")

    question.append(
        "There were nine computers in the server room. Five more computers "
        "were installed each day, from monday to thursday. "
        "How many computers are now in the server room?"
    )
    chain.append(
        "There were originally 9 computers. For each of 4 days, 5 more "
        "computers were added. So 5 * 4 = 20 computers were added. "
        "9 + 20 is 29."
    )
    answer.append("29")

    question.append(
        "Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On "
        "wednesday, he lost 2 more. "
        "How many golf balls did he have at the end of wednesday?"
    )
    chain.append(
        "Michael started with 58 golf balls. After losing 23 on tuesday, "
        "he had 58 - 23 = 35. After losing 2 more, "
        "he had 35 - 2 = 33 golf balls."
    )
    answer.append("33")

    question.append(
        "Olivia has $23. She bought five bagels for $3 each. "
        "How much money does she have left?"
    )
    chain.append(
        "Olivia had 23 dollars. "
        "5 bagels for 3 dollars each will be 5 x 3 = 15 dollars. "
        "So she has 23 - 15 dollars left. 23 - 15 is 8."
    )
    answer.append("8")

    # randomize order of the examples ...
    index_list = list(range(len(question)))
    random.shuffle(index_list)

    # Concatenate demonstration examples ...
    demo_text = ""
    for i in index_list[:n_shot]:
        if cot_flag:
            demo_text += (
                "Q: "
                + question[i]
                + "\nA: "
                + chain[i]
                + " "
                + ANSWER_TRIGGER
                + " "
                + answer[i]
                + ".\n\n"
            )
        else:
            demo_text += (
                "Question: "
                + question[i]
                + "\nAnswer: "
                + ANSWER_TRIGGER
                + " "
                + answer[i]
                + ".\n\n"
            )
    return demo_text

In [11]:
def build_prompt(input, n_shot, cot_flag):
    demo = create_demo_text(n_shot, cot_flag)
    input_text_prompt = demo + "Q: " + input['question'] + "\n" + "A: "
    return input_text_prompt

# **LLaMA 405B API Inference to get Logits and Top_k**

In [ ]:
import os
import openai

# client = openai.OpenAI(
#   api_key='916fe265dcbdc747eaef2fce05f10dea0178aa651b160c861a426af751028dc3', # Together AI
#   base_url="https://api.together.xyz/v1",
# )

client = openai.OpenAI(
    api_key=###,
    base_url="https://api.hyperbolic.xyz/v1",
    )

CONTEXT = f"You are an expert in solving math questions. Here are some examples:\n {create_demo_text(2, COT_FLAG)} \n Solve the following question step by step and use \"The answer is:\" to provide your answer."

def get_405B_answer(input, context=CONTEXT):
  prompt = "Q: " + input['question'] + "\n" + "A: "
  response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-405B-Instruct",
    messages=[
        {"role": "system", "content": context},
        {"role": "user", "content": prompt},
    ],
    temperature=0.8,
    max_tokens=300,
    # logprobs=True,
    # top_logprobs=5,
    )

  return response

In [13]:
def parse_response(row):
    response = get_405B_answer(row, context=CONTEXT)
    choice = response.choices[0]
    answer_text = choice.message.content
    # logprobs_list = choice.logprobs.content  # List of token logprobs per token

    # Combine extraction of tokens and logprobs in one pass
    # combined = [
    #     (
    #         [tp.token for tp in token_item.top_logprobs],
    #         [tp.logprob for tp in token_item.top_logprobs]
    #     )
    #     for token_item in logprobs_list
    # ]

    # Unzip the combined list into separate lists; handle empty case
    # if combined:
    #     token_predictions, token_logprobs = map(list, zip(*combined))
    # else:
    #     token_predictions, token_logprobs = [], []

    row["405B_answer"] = answer_text
    # row["405B_predictions"] = token_predictions
    # row["405B_logprobs"] = token_logprobs

    return row

In [14]:
print(parse_response(test_405[0]))

{'question': "Darrell and Allen's ages are in the ratio of 7:11. If their total age now is 162, calculate Allen's age 10 years from now.", 'answer': "The total ratio representing their ages is 7+11= <<7+11=18>>18\nSince the fraction of the ratio that represents Allen's age is 11/18, Allen's current age is 11/18*162 = <<11/18*162=99>>99\nIf Allen is currently 99 years old, in 10 years he will be 99+10 = <<99+10=109>>109 years old\n#### 109", '405B_answer': "To solve this problem, we'll break it down into steps.\n\nStep 1: Let's represent Darrell and Allen's ages using the given ratio. We'll use 7x for Darrell's age and 11x for Allen's age.\n\nStep 2: The total age of Darrell and Allen now is 162. We can write the equation: 7x + 11x = 162.\n\nStep 3: Combine like terms in the equation: 18x = 162.\n\nStep 4: Now, solve for x by dividing both sides by 18: x = 162 / 18 = 9.\n\nStep 5: Now that we know x is 9, we can find Allen's current age: Allen's age = 11x = 11 * 9 = 99.\n\nStep 6: To fi

In [15]:
test_405_answers = test_405.map(parse_response)
test_405_answers.save_to_disk("./405_answers")

Parameter 'function'=<function parse_response at 0x7ac65f0cd4e0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/200 [00:00<?, ? examples/s]

**Create new dataset with LLaMA 405B responses (run once)**

In [ ]:
# Adds entries to the two columns
new_dataset = gsm8k_kd_sample_new.map(parse_response)

new_dataset.save_to_disk("./gsm8k_kd_sample_new")

Parameter 'function'=<function parse_response at 0x7b83f81304a0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/400 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_from_disk, concatenate_datasets
dataset1 = load_from_disk("./gsm8k_kd_sample")
dataset2 = load_from_disk("./gsm8k_kd_sample_new")

merged_dataset = concatenate_datasets([dataset1, dataset2])
merged_dataset.save_to_disk("./gsm8k_kd_800")

Saving the dataset (0/1 shards):   0%|          | 0/800 [00:00<?, ? examples/s]

# **Knowledge Distillation**

**Load the saved dataset**

In [ ]:
from datasets import load_from_disk

kd_data = load_from_disk("./gsm8k_kd_800")

In [ ]:
print(kd_data[0])

{'question': 'Mimi picked up 2 dozen seashells on the beach.  Kyle found twice as many shells as Mimi and put them in his pocket. Leigh grabbed one-third of the shells that Kyle found.  How many seashells did Leigh have?', 'answer': 'Mimi has 2 x 12 = <<2*12=24>>24 sea shells.\nKyle has 24 x 2 = <<24*2=48>>48 sea shells.\nLeigh has 48 / 3 = <<48/3=16>>16 sea shells.\n#### 16', '405B_answer': "To solve this problem, let's break it down step by step.\n\nMimi picked up 2 dozen seashells. Since 1 dozen equals 12, 2 dozen equals 2 x 12 = 24.\n\nKyle found twice as many shells as Mimi, so Kyle found 2 x 24 = 48 seashells.\n\nLeigh grabbed one-third of the shells that Kyle found. Since Kyle found 48 seashells, Leigh grabbed 1/3 x 48 = 16 seashells.\n\nThe answer is: 16.", '405B_predictions': [['To', 'Let', 'First', 'M', 'Step'], [' solve', ' find', ' determine', ' start', ' answer'], [' this', ' the', ' for', ' Leigh', ' these'], [' problem', ',', ' question', ' step', ' problems'], [',', ' s

**Load Student Model**

In [ ]:
# LLaMA 3.2 3B
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B")
# model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B", device_map="auto")

# tokenizer.pad_token = tokenizer.eos_token
# model.config.pad_token_id = model.config.eos_token_id

special_tokens_dict = {"pad_token": "<pad>"}
num_added_tokens = tokenizer.add_special_tokens(special_tokens_dict)
if num_added_tokens > 0:
  model.resize_token_embeddings(len(tokenizer))

# tokenizer.pad_token = '<|finetune_right_pad_id|>'

# EOS_TOKEN = tokenizer.eos_token

model-00001-of-00002.safetensors:  19%|#9        | 944M/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=16,                       # LoRA rank
    lora_alpha=32,            # LoRA scaling
    lora_dropout=0.05,        # dropout for LoRA
    bias="none",              # LoRA doesn't add bias by default
    task_type=TaskType.CAUSAL_LM,  # for a causal language model
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"]
)

# Inject LoRA adapters into the original model
lora_model = get_peft_model(model, lora_config)

In [ ]:
lora_model.print_trainable_parameters()

trainable params: 4,587,520 || all params: 3,217,340,416 || trainable%: 0.1426


In [ ]:
import numpy as np
import torch

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# **Black box KD**

In [ ]:
def tokenize_kd(example):
    prompt_text = build_prompt(example, 2, COT_FLAG)
    kd = example["405B_answer"]
    full_text = prompt_text + kd

    # Tokenize separately
    prompt_enc = tokenizer(prompt_text, add_special_tokens=True)
    full_enc = tokenizer(
        full_text,
        truncation=True,
        padding="max_length",
        max_length=900,
        add_special_tokens=True,
    )

    # Copy labels
    input_ids = full_enc["input_ids"]
    labels = input_ids.copy()

    # Number of prompt tokens
    num_prompt_tokens = len(prompt_enc["input_ids"])

    # Mask out the prompt portion
    for i in range(len(input_ids)):
      if i in range(num_prompt_tokens) and i < len(labels):
        labels[i] = -100
      elif input_ids[i] == tokenizer.pad_token_id and i < len(labels):
        labels[i] = -100
        full_enc["attention_mask"][i] = 0

    # Put back in dictionary
    full_enc["labels"] = labels
    return full_enc

In [ ]:
kd_train = kd_data.map(tokenize_kd)
kd_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [ ]:
print(tokenizer.decode(kd_train[45]["input_ids"], skip_special_tokens=True))

Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is:  39.

Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8. The answer is:  8.

Q: Mr. Langsley commutes to work every day by bus. The bus picks him up at 6:00 a.m. and takes forty minutes to arrive at the first station. If Mr. Langsley arrives at work at 9:00 a.m., what's the total time taken in minutes from the first station to Mr. Langsley's workplace?
A: To find the total time taken from the first station to Mr. Langsley's workplace, we first need to find the total time taken for his entire commute. 

The bus picks him up at 6:00 a.m. and Mr. Langsl

In [ ]:
output_dir = "./gsm8k_bbkd"

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

# Data collator for dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,  # start small to avoid OOM
    gradient_accumulation_steps=16,  # effective batch size = 4
    num_train_epochs=5,            # set higher if you have the compute
    learning_rate=2e-4,            # LoRA can often use a slightly higher LR
    logging_steps=5,
    save_steps=50,
    fp16=True,                     # if on GPU that supports half precision
    gradient_checkpointing=True,
    logging_dir="./logs",
    optim="adamw_torch",
    weight_decay=0.01, # do 0.005 or 0.001?
    warmup_steps=25
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=kd_train,
    # eval_dataset=eval_dataset,
    data_collator=data_collator
)

trainer.train()
trainer.save_model(output_dir)

Step,Training Loss
5,0.435200
10,0.402300
15,0.407400
20,0.386600
25,0.438400
30,0.422300
35,0.446700
40,0.441600
45,0.420900
50,0.444400


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetunin

# **White Box KD**

In [ ]:
# This function returns the token_id of a given token and handles any errors

def convert_token(token, tokenizer):
    token_stripped = token.strip()
    if not token_stripped:
        return tokenizer.unk_token_id if tokenizer.unk_token_id is not None else tokenizer.eos_token_id
    token_id = tokenizer.convert_tokens_to_ids(token_stripped)
    if token_id is None or token_id < 0:
        return tokenizer.unk_token_id if tokenizer.unk_token_id is not None else tokenizer.eos_token_id
    return token_id

In [ ]:
print(kd_data[0]["405B_logprobs"])

[[-0.15001878142356873, -2.1792850494384766, -4.208555221557617, -4.520750045776367, -7.798799514770508], [-1.0533314943313599, -0.42894187569618225, -10.57528305053711, -13.541135787963867, -13.853330612182617], [-0.19056373834609985, -1.751537799835205, -20.483245849609375, -21.107635498046875, -21.497879028320312], [-0.0890183076262474, -2.5865769386291504, -4.615847110748291, -16.79145622253418, -17.259748458862305], [0.0, -18.731708526611328, -27.47317123413086, -27.78536605834961, -27.941463470458984], [-0.6931484937667847, -0.6931484937667847, -14.507782936096191, -14.976075172424316, -15.054123878479004], [0.0, -31.063413619995117, -32.31219482421875, -32.93658447265625, -35.278045654296875], [-0.0551212802529335, -3.1770694255828857, -4.5819501876831055, -6.6112165451049805, -8.640484809875488], [0.0, -23.414634704589844, -26.224390029907227, -33.248779296875, -33.8731689453125], [0.0, -23.414630889892578, -28.409751892089844, -30.126827239990234, -32.312191009521484], [-0.017

In [ ]:
# This function tokenizes the prompts as needed and builds the input dictionary for training; the first half should be based on whatever tokenizing function
# you are using originally for the dataset, with the correct prompts and helper functions if needed.

def tokenize_soft(example):
    prompt_text = build_prompt(example, 2, COT_FLAG)
    reasoning_text = example["answer"].split("#### ")[0]
    answer = example["answer"].split("#### ")[1]
    full_text = prompt_text + reasoning_text + "The answer is: " + answer + "."

    # Tokenize separately
    prompt_enc = tokenizer(prompt_text, add_special_tokens=True)
    full_enc = tokenizer(
        full_text,
        truncation=True,
        padding="max_length",
        max_length=900,
        add_special_tokens=True,
    )

    # Copy labels
    input_ids = full_enc["input_ids"]
    labels = input_ids.copy()

    # Number of prompt tokens
    num_prompt_tokens = len(prompt_enc["input_ids"])

    # Mask out the prompt portion
    for i in range(len(input_ids)):
      if i in range(num_prompt_tokens) and i < len(labels):
        labels[i] = -100
      elif input_ids[i] == tokenizer.pad_token_id and i < len(labels):
        labels[i] = -100
        full_enc["attention_mask"][i] = 0

    # Put back in dictionary
    full_enc["labels"] = labels

    # EVERYTHING ABOVE SHOULD BE YOUR NORMAL TOKENIZER FUNCTION; adapt for different datasets
    # The following adds two more columns (or two more keys to the dictionary)

    # Convert each of the tokens in the list of lists to token_ids
    teacher_preds_ids = [
    [convert_token(token, tokenizer) for token in token_list]
    for token_list in example["405B_predictions"]
    ]

    full_enc["teacher_predictions"] = teacher_preds_ids
    full_enc["teacher_logprobs"] = example["405B_logprobs"]
    return full_enc

**Note: It is assumed that "kd_data" has been previously loaded as a dataset, and contains the columns used in the above function by example["key"]**

In [ ]:
distill_train = kd_data.map(tokenize_soft)
# This builds dictionary of tensors
distill_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels", "teacher_predictions", "teacher_logprobs"])

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [ ]:
print(kd_data[0]["405B_predictions"])

[['To', 'Let', 'First', 'M', 'Step'], [' solve', ' find', ' determine', ' start', ' answer'], [' this', ' the', ' for', ' Leigh', ' these'], [' problem', ',', ' question', ' step', ' problems'], [',', ' step', '.', ' let', ':\n\n'], [' let', ' we', ' first', ' Let', ' lets'], ["'s", "'", " '", ' us', '’s'], [' break', ' start', ' follow', ' first', ' do'], [' it', ' down', ' It', ' is', ' in'], [' down', ' step', ' into', ' up', ' Down'], [' step', ' into', ':\n\n', ' steps', ' one'], [' by', '-by', ' By', '.by', '_by'], [' step', '-step', ' Step', ' steps', 'step'], ['.\n\n', ':\n\n', '.', ':\n', ':'], ['M', '1', 'First', 'Step', 'Since'], ['imi', 'im', 'imo', 'umi', 'iri'], [' picked', ' has', ' found', ' started', ' initially'], [' up', ' ', '-up', ' Up', ' out'], [' ', ' "', " '", ' two', ' se'], ['2', '1', '۲', ' ', '3'], [' dozen', ' dozens', '-do', ' doz', ' do'], [' se', ' (', ' sea', ' shells', ' seaw'], ['ash', 'ashes', 'ah', 'ach', 'аш'], ['ells', 'ell', 'els', 'ulls', 'alls

In [ ]:
print(distill_train[0]["teacher_predictions"])

tensor([[  1271,  10267,   5451,     44,   8468],
        [ 60519,   3990, 128001,   2527,   9399],
        [   576,   1820,   2000, 128001,  45010],
        [ 35686,     11,   7998,   9710,  96440],
        [    11,   9710,     13,   1169,     25],
        [  1169,    906,   3983,  10267,  10145],
        [   596,      6,      6,    355, 128001],
        [  9137,   2527,  19070,   3983,   3055],
        [   275,   2996,   2181,    285,    258],
        [  2996,   9710,  18614,    455,   4554],
        [  9710,  18614,     25,  25047,    606],
        [  1729,  14656,   1383,  19544,   3795],
        [  9710,  30308,   8468,  25047,   9710],
        [    13,     25,     13,     25,     25],
        [    44,     16,   5451,   8468,  12834],
        [ 25877,    318,  11620,  25330,  21336],
        [ 94599,   4752,  15456,  47823, 128001],
        [   455, 128001,   5352,   2378,    412],
        [128001,      1,      6,  20375,    325],
        [    17,     16, 128001, 128001,     18],


In [ ]:
print(distill_train[0]["teacher_logprobs"])

tensor([[-1.5002e-01, -2.1793e+00, -4.2086e+00, -4.5208e+00, -7.7988e+00],
        [-1.0533e+00, -4.2894e-01, -1.0575e+01, -1.3541e+01, -1.3853e+01],
        [-1.9056e-01, -1.7515e+00, -2.0483e+01, -2.1108e+01, -2.1498e+01],
        [-8.9018e-02, -2.5866e+00, -4.6158e+00, -1.6791e+01, -1.7260e+01],
        [ 0.0000e+00, -1.8732e+01, -2.7473e+01, -2.7785e+01, -2.7941e+01],
        [-6.9315e-01, -6.9315e-01, -1.4508e+01, -1.4976e+01, -1.5054e+01],
        [ 0.0000e+00, -3.1063e+01, -3.2312e+01, -3.2937e+01, -3.5278e+01],
        [-5.5121e-02, -3.1771e+00, -4.5820e+00, -6.6112e+00, -8.6405e+00],
        [ 0.0000e+00, -2.3415e+01, -2.6224e+01, -3.3249e+01, -3.3873e+01],
        [ 0.0000e+00, -2.3415e+01, -2.8410e+01, -3.0127e+01, -3.2312e+01],
        [-1.7127e-02, -4.0757e+00, -2.0310e+01, -2.2183e+01, -2.3276e+01],
        [ 0.0000e+00, -1.8107e+01, -3.5434e+01, -3.9180e+01, -4.0585e+01],
        [ 0.0000e+00, -2.3727e+01, -2.4039e+01, -2.4820e+01, -2.5132e+01],
        [-3.7726e-01, -1.

**The following defines the white-box KD training instance with hyperparameters alpha and beta. The value alpha determines the ratio of forward KLD (vs. reverse KLD) and the value beta determines the ratio of ground truth loss (vs. total KLD).**

In [ ]:
import torch
import torch.nn.functional as F
from transformers import Trainer

# def recursive_convert_to_float(obj):
#     if isinstance(obj, list):
#         return [recursive_convert_to_float(x) for x in obj]
#     return float(obj)

# Helper function to compute teacher probability distribution from teacher_logprobs.
def compute_teacher_distribution(teacher_logprobs):
    # If teacher_logprobs is a list, convert it to a tensor.
    # if isinstance(teacher_logprobs, list):
    #     teacher_logprobs = recursive_convert_to_float(teacher_logprobs)
    #     teacher_logprobs = torch.tensor(teacher_logprobs, dtype=torch.float)
    # Convert negative log probs to probs (p = exp(-nlogp)), then normalize over the top-k tokens. (shape: [B, S, 5])
    teacher_probs = torch.exp(-teacher_logprobs)
    teacher_probs = teacher_probs / teacher_probs.sum(dim=-1, keepdim=True)
    return teacher_probs

class DistillationTrainer(Trainer):
    def __init__(self, *args, alpha=0.5, beta=0.5, **kwargs):
        """
        alpha: weight for the distillation (KL) loss.
        beta: weight for the ground truth (CE) loss.
        """
        super().__init__(*args, **kwargs)
        self.alpha = alpha  # weight for forward vs. reverse KL loss
        self.beta = beta    # weight for the ground-truth CE loss vs KL

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # Extract teacher predictions and logprobs from the inputs and remove them from the dict.
        teacher_preds = inputs.pop("teacher_predictions")  # expected shape: [B, seq_length, top_k]
        teacher_logprobs = inputs.pop("teacher_logprobs")   # expected shape: [B, seq_length, top_k]

        # model_inputs = {key: inputs[key] for key in ["input_ids", "attention_mask", "labels"] if key in inputs}
        outputs = model(**inputs)
        logits = outputs.logits  # shape: [B, seq_length, vocab_size]

        # 1. Compute cross-entropy loss with the ground truth.
        labels = inputs.get("labels")
        loss_ce = F.cross_entropy(
            logits.view(-1, logits.size(-1)),
            labels.view(-1),
            ignore_index=-100
        )

        # 2. Compute the distillation loss on the teacher’s token predictions.
        # Convert teacher_logprobs into a probability distribution.
        teacher_probs = compute_teacher_distribution(teacher_logprobs)  # shape: [B, seq_length, top_k]

        # Gather the student logits for the teacher's top-k tokens.
        # teacher_preds should contain the token indices corresponding to the top-k teacher predictions.
        student_logits_top = torch.gather(logits, dim=-1, index=teacher_preds)
        student_probs_top = F.softmax(student_logits_top, dim=-1)

        # Compute forward KL: KL(teacher || student) = Σ teacher_probs * (log(teacher_probs) - log(student_probs_top))
        forward_kl = (teacher_probs * (torch.log(teacher_probs + 1e-8) - torch.log(student_probs_top + 1e-8))).sum(dim=-1).mean()

        # Compute reverse KL: KL(student || teacher) = Σ student_probs_top * (log(student_probs_top) - log(teacher_probs))
        reverse_kl = (student_probs_top * (torch.log(student_probs_top + 1e-8) - torch.log(teacher_probs + 1e-8))).sum(dim=-1).mean()

        # Combine the two KL terms.
        distill_loss = self.alpha * forward_kl + (1 - self.alpha) * reverse_kl

        # Combine the losses.
        loss = (1 - self.beta) * distill_loss + self.beta * loss_ce

        if return_outputs:
            return loss, outputs
        return loss


**We should run the following experiments: \
(alpha, beta) = (0.5, 0.5) which is the default \
(alpha, beta) = (0, 0.5) which is fully reverse KLD (mode-seeking) \
(alpha, beta) = (1, 0.5) which is fully foward KLD (mode-covering). \
If time permits, we can explore other options, such as decreasing beta and asymetrical combinations of KLD.**

In [ ]:
alpha = 0.5  # Weight for forward KL; (1-alpha) for reverse KL loss
beta = 0.5  # Weight for the ground truth (CE) loss; (1-beta) for total KL loss

output_dir = output_dir = f"./gsm8k_alpha={alpha}_beta={beta}" # saves in a different dir for different params

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

# Data collator for dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

# Set up training arguments.
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1, # Increasing batch size might cuase problems
    gradient_accumulation_steps=16,
    num_train_epochs=5,
    logging_steps=5, # adjust as needed
    save_steps=50, # adjust as needed
    learning_rate=1e-4,
    weight_decay=0.01,
    fp16=False,                     # needs to be turned off!
    gradient_checkpointing=True,
    logging_dir="./logs",
    # optim="adamw_torch",
    warmup_steps=20,  #This should be around 5-10% of total steps
    remove_unused_columns=False,  # to include additional columns
    report_to=["wandb"]
)

# Instantiate the custom distillation trainer.
trainer = DistillationTrainer(
    model=lora_model,
    args=training_args,
    train_dataset=distill_train,  # prepared dataset here
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Start training.
trainer.train()
trainer.save_model(output_dir)

<ipython-input-104-a508342b57b8>:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `DistillationTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


AssertionError: No inf checks were recorded for this optimizer.

# **Rank Optimization**

In [ ]:
import torch
import torch.nn.functional as F
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, AutoTokenizer

# Hyperparameters for ranking distillation; larger means stricter ordering
RANKING_MARGIN = 0.8  # Desired margin between scores for preferred vs. less-preferred tokens
LAM = 0.6 # ratio of CE Loss w true label

def compute_ranking_loss(student_logits_selected, margin=RANKING_MARGIN):
    """
    Compute a pairwise ranking loss for a tensor of student logits that have been gathered
    for the teacher's top-K tokens.

    Args:
        student_logits_selected (torch.Tensor): shape [B, S, K] where B=batch size, S=sequence length,
                                                  K=number of teacher-ranked tokens (e.g., 5).
        margin (float): The margin to enforce between logits for tokens ranked higher vs. lower.

    Returns:
        torch.Tensor: Scalar ranking loss.
    """
    B, S, K = student_logits_selected.shape
    loss = 0.0
    count = 0
    # For each pair of positions in the teacher's ranked list:
    # Teacher implies that token at position i should have a higher logit than token at position j if i < j.
    for i in range(K):
        for j in range(i+1, K):
            # Difference between the logit for the token ranked i and the one ranked j.
            diff = student_logits_selected[..., i] - student_logits_selected[..., j]  # shape: [B, S]
            # We want diff >= margin. If not, incur a hinge loss.
            pair_loss = F.relu(margin - diff)  # shape: [B, S]
            loss += pair_loss.mean()  # average loss over batch and sequence for this pair
            count += 1

    return loss / count if count > 0 else loss

class RankingDistillationTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kargs):
        # Extract teacher's ranked token indices.
        # Expected shape: [B, S, K]
        teacher_top_indices = inputs.pop("teacher_predictions")
        inputs.pop("teacher_logprobs")

        # Forward pass: compute the student model's logits.
        outputs = model(**inputs)
        student_logits = outputs.logits  # shape: [B, S, V] where V is the vocabulary size

        # Compute standard cross entropy loss.
        # Here, we flatten batch and sequence dimensions.
        labels = inputs.get("labels")
        ce_loss = F.cross_entropy(
            student_logits.view(-1, student_logits.size(-1)),
            labels.view(-1),
            ignore_index=-100  # Adjust if you're using a different padding/ignore index.
        )

        # Gather student logits for the teacher's top tokens.
        # teacher_top_indices is used to index the vocabulary dimension.
        student_logits_selected = torch.gather(student_logits, dim=-1, index=teacher_top_indices)
        # student_logits_selected has shape [B, S, K]
        # Compute the ranking loss.
        ranking_loss = compute_ranking_loss(student_logits_selected, margin=RANKING_MARGIN)

        # Compute the total loss.
        loss = LAM * ce_loss + (1-LAM) * ranking_loss

        return (loss, outputs) if return_outputs else loss

In [ ]:
output_dir = f"./gesm8k_rank_margin={RANKING_MARGIN}_lam={LAM}"

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

# Data collator for dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

# Set up training arguments.
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1, # Increasing batch size might cuase problems
    gradient_accumulation_steps=16,
    num_train_epochs=3,
    logging_steps=5, # adjust as needed
    save_steps=50, # adjust as needed
    learning_rate=2e-4,
    weight_decay=0.01,
    fp16=False,                     # needs to be turned off!
    gradient_checkpointing=True,
    logging_dir="./logs",
    # optim="adamw_torch",
    warmup_steps=20,  #This should be around 5-10% of total steps
    remove_unused_columns=False,  # to include additional columns
    report_to=["wandb"]
)

# Instantiate the custom distillation trainer.
trainer = RankingDistillationTrainer(
    model=model,
    args=training_args,
    train_dataset=distill_train,  # prepared dataset here
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Start training.
trainer.train()
trainer.save_model(output_dir)

<ipython-input-108-cac5a28c33fb>:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `RankingDistillationTrainer.__init__`. Use `processing_class` instead.
  trainer = RankingDistillationTrainer(


Step,Training Loss
5,4.762700
10,4.579900
15,4.487000
20,4.388400
25,4.371100
30,4.347800
35,4.326300
40,4.287000
45,4.322800
50,4.290400


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetunin

In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.optim import Adam

inputs = distill_train[0]
labels = inputs["labels"]

# Define an optimizer
optimizer = Adam(model.parameters(), lr=1e-4)

# Ensure model is in training mode
lora_model.train()

# --- Forward Pass and Loss Computation ---
# Perform a forward pass; the model returns logits.
outputs = lora_model(**inputs, labels=labels)
loss = outputs.loss

print("Loss before backward:", loss.item())

# --- Backward Pass: Compute Gradients ---
optimizer.zero_grad()
loss.backward()

# --- 1. Inspect Gradients: Check each parameter's .grad attribute ---
print("\nChecking gradients for each parameter:")
for name, param in model.named_parameters():
    if param.requires_grad:
        if param.grad is None:
            print(f"Parameter '{name}' has NO gradient!")
        else:
            grad_norm = param.grad.norm().item()
            print(f"Parameter '{name}' gradient norm: {grad_norm:.4f}")

# --- 2. Ensure that all model parameters have requires_grad=True ---
print("\nVerifying requires_grad for each parameter:")
for name, param in model.named_parameters():
    print(f"Parameter '{name}' requires_grad: {param.requires_grad}")

# --- 3. Verify Parameter Updates ---
# Save a copy of current parameters
old_params = {name: param.clone() for name, param in model.named_parameters() if param.requires_grad}

# Perform an optimizer step to update parameters
optimizer.step()

print("\nVerifying parameter updates:")
for name, param in model.named_parameters():
    if param.requires_grad:
        # Check if the parameter has changed
        if torch.equal(param, old_params[name]):
            print(f"Parameter '{name}' did NOT update!")
        else:
            print(f"Parameter '{name}' updated.")



# **Full Fine Tuning:**

In [ ]:
print(gsm8k_train[0])

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}


**The 2 in build_prompt means we're using 2-shot examples. Can be changed to up to 8. Do not put 0.**

In [ ]:
def tokenize_fn(example):
    prompt_text = build_prompt(example, 2, COT_FLAG)
    reasoning_text = example["answer"].split("#### ")[0]
    answer = example["answer"].split("#### ")[1]
    full_text = prompt_text + reasoning_text + "The answer is: " + answer + "."

    # Tokenize separately
    prompt_enc = tokenizer(prompt_text, add_special_tokens=True)
    full_enc = tokenizer(
        full_text,
        truncation=True,
        padding="max_length",
        max_length=512,
        add_special_tokens=True,
    )

    # Copy labels
    input_ids = full_enc["input_ids"]
    labels = input_ids.copy()

    # Number of prompt tokens
    num_prompt_tokens = len(prompt_enc["input_ids"])

    # Mask out the prompt portion
    for i in range(len(input_ids)):
      if i in range(num_prompt_tokens) and i < len(labels):
        labels[i] = -100
      elif input_ids[i] == tokenizer.pad_token_id and i < len(labels):
        labels[i] = -100
        full_enc["attention_mask"][i] = 0

    # Put back in dictionary
    full_enc["labels"] = labels
    return full_enc


**Run this for ground truth labels and finetuning**

In [ ]:
train_dataset = kd_data.map(tokenize_fn)
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [ ]:
# Activate this for further train eval split.
# split_dataset = train_dataset.train_test_split(test_size=0.1, seed=42)
# train_dataset = split_dataset["train"]
# eval_dataset = split_dataset["test"]

In [ ]:
print(train_dataset[0])

{'input_ids': tensor([128000,     48,     25,  ..., 128256, 128256, 128256]), 'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0]), 'labels': tensor([-100, -100, -100,  ..., -100, -100, -100])}


In [ ]:
print(tokenizer.decode(train_dataset[3]["input_ids"], skip_special_tokens=True))

Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is:  39.

Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
A: Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. The answer is:  9.

Q: Emma's bank account has $100 in it. Each day of the week, she spends $8. At the end of the week, she goes to the bank and asks for as many $5 bills as her account can give her. She leaves the rest in the account. How many dollars remain in the account?
A: She spend $56 because 7 x 8 = <<7*8=56>>56
She has $44 left in the bank because 100 - 56 = <<100-56=44>>44
She can get 8 five dollar bills because 44 / 5 = <<44/5=8.8>>8.8
This is equal to $40 because 8 x 5 = <<8*5=40>>40
She has $4 

**Finetuning**

In [ ]:
sample = train_dataset[random.randint(0, 800)]
print(sample["input_ids"])
print("Max token id:", max(sample["input_ids"]))
print("Min token id:", min(sample["input_ids"]))
print("Vocab size:", tokenizer.vocab_size)


tensor([128000,     48,     25,   2684,    527,    220,    868,  12690,    304,
           279,  10707,    588,     13,  41234,   7487,    690,   6136,  12690,
           304,    279,  10707,    588,   3432,     13,   4740,    814,    527,
          2884,     11,   1070,    690,    387,    220,   1691,  12690,     13,
          2650,   1690,  12690,   1550,    279,  10707,    588,   7487,   6136,
          3432,   5380,     32,     25,   2684,    527,    220,    868,  12690,
         13517,     13,   5112,   1070,   1051,    220,   1691,  12690,   1306,
          1063,    810,   1051,  39441,     13,   2100,   1070,   2011,    617,
          1027,    220,   1691,    482,    220,    868,    284,    220,     21,
            13,    578,   4320,    374,     25,    220,    220,     21,    382,
            48,     25,   2684,   1051,  11888,  19002,    304,    279,   3622,
          3130,     13,  21594,    810,  19002,   1051,  10487,   1855,   1938,
            11,    505,  94299,    311, 

**Remember to change ouptut_dir and save path in all fields.**

In [ ]:
output_dir = "./gsm8k_ft800"

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

# Data collator for dynamic padding
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=3,  # start small to avoid OOM
    gradient_accumulation_steps=16,  # effective batch size = 4
    num_train_epochs=3,            # set higher if you have the compute
    learning_rate=1e-4,            # LoRA can often use a slightly higher LR
    logging_steps=1,
    save_steps=40,
    fp16=True,                     # if on GPU that supports half precision
    gradient_checkpointing=True,
    logging_dir="./logs",
    optim="adamw_torch",
    weight_decay=0.01, # do 0.005 or 0.001?
    warmup_steps=5
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_dataset,
    # eval_dataset=eval_dataset,
    # data_collator=data_collator
)

trainer.train()
trainer.save_model(output_dir)

Step,Training Loss
1,20.700700
2,20.699300
3,19.449500
4,16.833800
5,14.002400
6,11.353700
7,9.106600
8,8.781000
9,8.463400
10,8.061800


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


# **Eval**

**Load Lora Model**

In [ ]:
eval_path = output_dir

In [ ]:
print(eval_path)

./gsm8k_ft800


In [ ]:
from operator import ge
from transformers import pipeline
from peft import PeftModel

# Replace "./fine_tuned_model" with the path to your fine-tuned model
adapted_model = PeftModel.from_pretrained(model, eval_path)
merged_model = adapted_model.merge_and_unload()
gen_pipe = pipeline("text-generation", model=merged_model, tokenizer=tokenizer)

Device set to use cuda:0


In [16]:
def clean_answer(model_pred, n_shot):
    model_pred = model_pred.lower()
    preds = model_pred.split(ANSWER_TRIGGER.lower())
    answer_flag = True if len(preds) > n_shot + 1 else False
    if answer_flag:
        # Pick first answer with flag
        pred = preds[n_shot + 1]
    else:
        # Pick last number without flag
        pred = preds[-1]

    pred = pred.replace(",", "")
    pred = [s for s in re.findall(r"-?\d+\.?\d*", pred)]

    if len(pred) == 0:
        return INVALID_ANS

    if answer_flag:
        # choose the first element in list
        pred = pred[0]
    else:
        # choose the last element in list
        pred = pred[-1]

    # (For arithmetic tasks) if a word ends with period, it will be omitted ...
    if pred[-1] == ".":
        pred = pred[:-1]

    return pred

In [17]:
def extract_answer_from_output(completion):
    match = ANS_RE.search(completion)
    if match:
        match_str = match.group(1).strip()
        match_str = match_str.replace(",", "")
        return match_str
    else:
        return INVALID_ANS

In [ ]:
# example test
question = gsm8k_test[7]["question"]
demo = create_demo_text(2, COT_FLAG)
prompt = demo + "Q: " + question + "\n" + "A: "
outputs = gen_pipe(prompt, max_new_tokens=256, top_p=0.95, temperature=0.8)
print(outputs[0]["generated_text"])

Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
A: Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. The answer is:  9.

Q: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?
A: There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. The answer is:  29.

Q: Carla is downloading a 200 GB file. Normally she can download 2 GB/minute, but 40% of the way through the download, Windows forces a restart to install updates, which takes 20 minutes. Then Carla has to restart the download from the beginning. How load does it take to download the file?
A: 200 GB is 200,000,000 MB. 2 GB / minute is 2,000,000,000 B / minute. 40% of the way through the download is 80,000,000 B. 80,000,000 B + 

**EVAL**

In [6]:
test_dataset = gsm8k_test.shuffle(seed=42).select(range(200))

In [ ]:
input_text = build_prompt(test_dataset[0], 2, COT_FLAG)
generate_kwargs = dict(max_new_tokens=256, top_p=0.95, temperature=0.8)
merged_model.eval()
model_completion = generate(merged_model, tokenizer, input_text, generate_kwargs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
print(model_completion)

In [18]:
def generate(model, tokenizer, input_text, generate_kwargs):
    input_text = tokenizer(
        input_text,
        padding=False,
        add_special_tokens=True,
        return_tensors="pt",
    )
    input_ids = input_text.input_ids.cuda()
    attention_mask = input_text.attention_mask.cuda()

    output_ids = model.generate(
        input_ids=input_ids, attention_mask=attention_mask, **generate_kwargs
    )
    response = []
    for i in range(output_ids.shape[0]):
        response.append(
            tokenizer.decode(
                output_ids[i][input_ids.shape[1] :],
                skip_special_tokens=True,
                ignore_tokenization_space=True,
            )
        )

    if len(response) > 1:
        return response
    return response[0]

In [23]:
answers = []
scores = []

for sample in tqdm(test_dataset):
  input_text = build_prompt(sample, 2, COT_FLAG)
  generate_kwargs = dict(max_new_tokens=256, top_p=0.95, temperature=0.8)
  model_completion = generate(model, tokenizer, input_text, generate_kwargs)
  # model_completion = sample["405B_answer"]
  model_answer = clean_answer(model_completion, 2)
  answers.append(model_answer)

  true_answer = extract_answer_from_output(sample["answer"])
  assert true_answer != INVALID_ANS

  if (model_answer == true_answer):
    scores.append(1)
  else:
    scores.append(0)

  0%|          | 0/200 [00:00<?, ?it/s]


NameError: name 'model' is not defined

In [ ]:
import pandas as pd

# Create a DataFrame with each list as a column
df = pd.DataFrame({
    "answers": answers,
    "correctness": scores
})

save_path = eval_path + "/eval_results.csv"

# Save the DataFrame to a new CSV file without the index column
df.to_csv(save_path, index=False)

In [22]:
print(
    f"Model type: 405B, "
    f"Num of total question: {len(scores)}, "
    f"Correct num: {sum(scores)}, "
    f"Accuracy: {float(sum(scores))/len(scores)}."
)

Model type: 405B, Num of total question: 200, Correct num: 176, Accuracy: 0.88.
